# Ollama + OpenAI + Python

## 1. تحديد اسم النموذج

إذا قمت بجلب نموذج مختلف عن "phi3:mini"، قم بتغيير القيمة في الخلية أدناه. سيتم استخدام هذا المتغير في الكود عبر الدفتر.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. إعداد عميل Open AI

عادةً ما يتم استخدام عميل OpenAI مع OpenAI.com أو Azure OpenAI للتفاعل مع نماذج اللغة الكبيرة.  
ومع ذلك، يمكن أيضًا استخدامه مع Ollama، حيث يوفر Ollama نقطة نهاية متوافقة مع OpenAI على "http://localhost:11434/v1".


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. إنشاء إكمال محادثة

الآن يمكننا استخدام OpenAI SDK لإنشاء استجابة لمحادثة. يجب أن ينتج هذا الطلب هايكو عن القطط:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. هندسة التوجيه

الرسالة الأولى التي يتم إرسالها إلى نموذج اللغة تُسمى "رسالة النظام" أو "توجيه النظام"، وهي تحدد التعليمات العامة للنموذج.  
يمكنك تقديم توجيه نظام مخصص خاص بك لتوجيه نموذج اللغة لإنتاج مخرجات بطريقة مختلفة.  
قم بتعديل `SYSTEM_MESSAGE` أدناه ليجيب مثل شخصية مشهورة من فيلم أو مسلسل تلفزيوني مفضل لديك، أو استلهم أفكارًا لتوجيهات نظام أخرى من [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts).  

بمجرد تخصيص رسالة النظام، قدم أول سؤال للمستخدم في `USER_MESSAGE`.  


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. أمثلة قليلة

طريقة أخرى لتوجيه نموذج اللغة هي تقديم "أمثلة قليلة"، وهي سلسلة من الأسئلة والأجوبة التي توضح كيفية الرد.

المثال أدناه يحاول جعل نموذج اللغة يتصرف كمساعد تدريس من خلال تقديم بعض الأمثلة للأسئلة والأجوبة التي قد يقدمها مساعد التدريس، ثم يقوم بتحفيز النموذج بسؤال قد يطرحه الطالب.

جربه أولاً، ثم قم بتعديل `SYSTEM_MESSAGE`، `EXAMPLES`، و `USER_MESSAGE` لسيناريو جديد.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. استرجاع المعلومات المدعوم بالتوليد

تقنية RAG (استرجاع المعلومات المدعوم بالتوليد) هي طريقة لجعل نموذج اللغة يجيب على الأسئلة بدقة في مجال معين، وذلك من خلال استرجاع المعلومات ذات الصلة أولاً من مصدر المعرفة ثم توليد إجابة بناءً على تلك المعلومات.

لقد قمنا بتوفير ملف CSV محلي يحتوي على بيانات حول السيارات الهجينة. يقوم الكود أدناه بقراءة ملف CSV، والبحث عن تطابقات مع سؤال المستخدم، ثم توليد إجابة بناءً على المعلومات التي تم العثور عليها. لاحظ أن هذا سيستغرق وقتًا أطول مقارنة بأي من الأمثلة السابقة، لأنه يرسل المزيد من البيانات إلى النموذج. إذا لاحظت أن الإجابة لا تزال غير مستندة إلى البيانات، يمكنك تجربة هندسة النظام أو تجربة نماذج أخرى. بشكل عام، تكون تقنية RAG أكثر فعالية مع النماذج الأكبر حجمًا أو مع الإصدارات التي تم تحسينها من نماذج SLM.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**إخلاء المسؤولية**:  
تم ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو معلومات غير دقيقة. يجب اعتبار المستند الأصلي بلغته الأصلية المصدر الرسمي. للحصول على معلومات حاسمة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة ناتجة عن استخدام هذه الترجمة.
